In [ ]:
pip install pdfplumber # pip install to get right system

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 70.5 MB/s eta 0:00:00


In [ ]:
pip install tqdm # pip install to get right system

In [ ]:
from tqdm import tqdm 

# **Extract Portion of Project**

In [3]:
import requests, zipfile, os
from io import BytesIO

def download_and_extract_ptr_zip(year=2025, out_dir="ptr_pdfs"):
    url = f"https://disclosures-clerk.house.gov/public_disc/financial-pdfs/{year}FD.zip"

    # Create output directory if it doesn't exist
    os.makedirs(out_dir, exist_ok=True)

    print(f"Downloading {url}...")
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to download: {url}")

    print("Extracting ZIP contents...")
    with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
        zip_ref.extractall(out_dir)

    print(f" Extracted to {out_dir}")
    return out_dir


In [ ]:
import os
import zipfile
from io import BytesIO
import requests

def download_and_extract_ptr_zip(year):
    url = f"https://disclosures-clerk.house.gov/public_disc/financial-pdfs/{year}FD.zip"
    out_dir = r"C:\Users\Edwin\Desktop\Project\ptr_pdfs"  

    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to download: {url}")

    print("Extracting ZIP contents...")
    with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
        zip_ref.extractall(out_dir)

    print(f" Extracted to {out_dir}")
    return out_dir


In [5]:
folder = download_and_extract_ptr_zip(2025)


Extracting ZIP contents...
✅ Extracted to C:\Users\Edwin\Desktop\Project\ptr_pdfs


## Now we are getting all the pdf file links from provided by the 2025 link so we have all the links to each pdf which has the ptr report for specific instance

In [ ]:
import pandas as pd

# Load the TXT file 
df_meta = pd.read_csv("C:\\Users\\Edwin\\Desktop\\Project\\ptr_pdfs\\2025FD.txt", sep="\t")

# Filter only PTR (Periodic Transaction Reports)
df_ptrs = df_meta[df_meta["FilingType"] == "P"]

# Build download URLs
df_ptrs["PDF_URL"] = df_ptrs["DocID"].astype(str).apply(
    lambda docid: f"https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/{docid}.pdf"
)

# Preview
print(df_ptrs[["First", "Last", "StateDst", "FilingDate", "PDF_URL"]].head())


              First         Last StateDst FilingDate  \
1        Richard W.        Allen     GA12  1/16/2025   
2        Richard W.        Allen     GA12  2/20/2025   
3        Richard W.        Allen     GA12  3/12/2025   
8              Jake  Auchincloss     MA04  2/21/2025   
12  Donald Sternoff        Beyer     VA08   1/6/2025   

                                              PDF_URL  
1   https://disclosures-clerk.house.gov/public_dis...  
2   https://disclosures-clerk.house.gov/public_dis...  
3   https://disclosures-clerk.house.gov/public_dis...  
8   https://disclosures-clerk.house.gov/public_dis...  
12  https://disclosures-clerk.house.gov/public_dis...  


C:\Users\Edwin\AppData\Local\Temp\ipykernel_43080\1866103336.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ptrs["PDF_URL"] = df_ptrs["DocID"].astype(str).apply(


## Now getting all the PDf's by going through each url made from the previous queries/ code this will then get each individual ptr and load it

In [ ]:
def download_ptr_pdfs(df, output_dir="ptr_pdfs", year=2025):
    os.makedirs(output_dir, exist_ok=True)

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Downloading PTR PDFs"):
        docid = str(row["DocID"]).strip()
        url = f"https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2025/{docid}.pdf"

        filename = os.path.join(output_dir, f"{docid}.pdf")

        if os.path.exists(filename):
            continue

        try:
            headers = {
                "User-Agent": "Mozilla/5.0",
                "Referer": "https://disclosures-clerk.house.gov/"
            }
            response = requests.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                with open(filename, "wb") as f:
                    f.write(response.content)
            else:
                print(f" Failed: {url} ({response.status_code})")
        except Exception as e:
            print(f" Error for {url}: {e}")



In [10]:
download_ptr_pdfs(df_ptrs)

# **Transform portion**

## Want to extract all the pdfs and make it into a dataframe

In [ ]:
import pdfplumber
import re
import pandas as pd

def extract_transactions_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            full_text = "\n".join([page.extract_text() or "" for page in pdf.pages])
    except Exception as e:
        print(f" Error reading {file_path}: {e}")
        return pd.DataFrame()  # Return empty DataFrame on failure

    # Match transaction lines that start with P, S, or S (partial)
    raw_chunks = re.split(r"\n(?=\w.*?(P|S(?: \(partial\))?)\s+\d{2}/\d{2}/\d{4})", full_text)

    records = []
    for chunk in raw_chunks:
        lines = chunk.strip().split("\n")
        try:
            asset_line = lines[0]
            transaction_line = lines[1]
            amount_line = lines[2] if "$" in lines[2] else transaction_line.split()[-1]
            source_keywords = ["IRA", "401K", "Trust", "Spouse", "Dependent", "S O:", "F S:"]
            source_line = " ".join([line for line in lines if any(kw in line for kw in source_keywords)])

            # Match P, S, or S (partial)
            match = re.search(r"(P|S(?: \(partial\))?)\s+(\d{2}/\d{2}/\d{4})\s+(\d{2}/\d{2}/\d{4})\s+(\$[\d,]+(?:\s*-\s*\$[\d,]+)?)", transaction_line)
            if match:
                trans_type, date, notify_date, amount = match.groups()
            else:
                trans_type = date = notify_date = amount = None

            records.append({
                "Asset": asset_line.strip(),
                "Transaction Type": trans_type,
                "Date": date,
                "Notification Date": notify_date,
                "Amount": amount,
                "Source": source_line.strip(),
                "File": file_path.split("/")[-1]
            })
        except:
            continue

    return pd.DataFrame(records)


## We are now looping through each file and make sure we get each one

In [12]:
import os

def extract_all_pdfs_to_dataframe(pdf_folder="ptr_pdfs"):
    all_dfs = []
    for file in os.listdir(pdf_folder):
        if file.endswith(".pdf"):
            file_path = os.path.join(pdf_folder, file)
            df = extract_transactions_from_pdf(file_path)
            if not df.empty:
                all_dfs.append(df)

    combined_df = pd.concat(all_dfs, ignore_index=True)
    return combined_df


# Running the functions

In [13]:
all_ptr_data = extract_all_pdfs_to_dataframe("ptr_pdfs")
print(all_ptr_data.head())
print(f"\n✅ Extracted {len(all_ptr_data)} transactions from {all_ptr_data['File'].nunique()} filings.")


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

                                               Asset Transaction Type  Date  \
0                                Filing ID #20018054             None  None   
1  Alphabet Inc. - Class C Capital Stock P 01/02/...             None  None   
2  Amazon.com, Inc. - Common Stock P 01/02/2025 0...             None  None   
3  Amazon.com, Inc. - Common Stock P 01/02/2025 0...             None  None   
4  Amphenol Corporation Common P 01/02/2025 01/03...             None  None   

  Notification Date Amount Source                   File  
0              None   None         ptr_pdfs\20018054.pdf  
1              None   None         ptr_pdfs\20018054.pdf  
2              None   None         ptr_pdfs\20018054.pdf  
3              None   None         ptr_pdfs\20018054.pdf  
4              None   None         ptr_pdfs\20018054.pdf  

✅ Extracted 2679 transactions from 166 filings.


# Verification of the load

In [14]:
all_ptr_data.head(100)


,Asset,Transaction Type,Date,Notification Date,Amount,Source,File
0,Filing ID #20018054,None,None,None,None,,ptr_pdfs\20018054.pdf
1,Alphabet Inc. - Class C Capital Stock P 01/02/...,None,None,None,None,,ptr_pdfs\20018054.pdf
2,"Amazon.com, Inc. - Common Stock P 01/02/2025 0...",None,None,None,None,,ptr_pdfs\20018054.pdf
3,"Amazon.com, Inc. - Common Stock P 01/02/2025 0...",None,None,None,None,,ptr_pdfs\20018054.pdf
4,Amphenol Corporation Common P 01/02/2025 01/03...,None,None,None,None,,ptr_pdfs\20018054.pdf
...,...,...,...,...,...,...,...
95,JT Intuit Inc. - Common Stock (INTU) S 01/13/2...,None,None,None,None,,ptr_pdfs\20024346.pdf
96,Lam Research Corporation - Common P 01/16/2025...,None,None,None,None,,ptr_pdfs\20024346.pdf
97,"JT Lamb Weston Holdings, Inc. Common S 01/13/2...",None,None,None,None,,ptr_pdfs\20024346.pdf
98,JT Linde plc - Ordinary Shares (LIN) S 01/13/2...,None,None,None,None,,ptr_pdfs\20024346.pdf


In [ ]:
# sending to csv and verifying with excel just to see all of the data at once
all_ptr_data.to_csv("all_ptr_data.csv", index=False)


In [16]:
import pandas as pd
import re

# Load the CSV
df = pd.read_csv("all_ptr_data.csv")

# Function to extract fields
def extract_transaction_info(asset_str):
    if not isinstance(asset_str, str):
        return pd.Series([None, None, None, None, asset_str])

    # Regex for full match of P, S, S (partial), Date, Notification Date, Amount
    match = re.search(
        r"\b(S \(partial\)|S|P)\s+(\d{2}/\d{2}/\d{4})\s+(\d{2}/\d{2}/\d{4})\s+(\$[\d,]+(?:\s*-\s*\$[\d,]+)?)",
        asset_str
    )

    if match:
        trans_type, date, notify_date, amount = match.groups()
        asset_cleaned = asset_str[:match.start()].strip()
    else:
        trans_type = date = notify_date = amount = None
        asset_cleaned = asset_str.strip()

    return pd.Series([asset_cleaned, trans_type, date, notify_date, amount])

# Apply extraction function
df[["Asset", "Transaction Type", "Date", "Notification Date", "Amount"]] = df["Asset"].apply(extract_transaction_info)

# Parse amount into min/max
def parse_amount_range(amount_str):
    if pd.isna(amount_str):
        return None, None
    matches = [int(val.replace(",", "")) for val in re.findall(r"\$([\d,]+)", amount_str)]
    if len(matches) == 1:
        return matches[0], matches[0]
    elif len(matches) == 2:
        return matches[0], matches[1]
    return None, None

df[["minimum", "maximum"]] = df["Amount"].apply(lambda x: pd.Series(parse_amount_range(x)))

# Final cleaned DataFrame
df_cleaned = df.copy()

# Save cleaned result
df_cleaned.to_csv("ptr_transactions_cleaned.csv", index=False)

# Show a check
print(df_cleaned["Transaction Type"].value_counts(dropna=False))



Transaction Type
P              1284
S               870
S (partial)     359
None            166
Name: count, dtype: int64


In [ ]:
# verifying structure of df_cleaned which is the dataframe which will be sent to postresql for further cleaning and analysis
df_cleaned


,Asset,Transaction Type,Date,Notification Date,Amount,Source,File,minimum,maximum
0,Filing ID #20018054,None,None,None,None,NaN,ptr_pdfs\20018054.pdf,NaN,NaN
1,Alphabet Inc. - Class C Capital Stock,P,01/02/2025,01/03/2025,"$1,001 - $15,000",NaN,ptr_pdfs\20018054.pdf,1001.0,15000.0
2,"Amazon.com, Inc. - Common Stock",P,01/02/2025,01/05/2025,"$1,001 - $15,000",NaN,ptr_pdfs\20018054.pdf,1001.0,15000.0
3,"Amazon.com, Inc. - Common Stock",P,01/02/2025,01/03/2025,"$1,001 - $15,000",NaN,ptr_pdfs\20018054.pdf,1001.0,15000.0
4,Amphenol Corporation Common,P,01/02/2025,01/03/2025,"$1,001 - $15,000",NaN,ptr_pdfs\20018054.pdf,1001.0,15000.0
...,...,...,...,...,...,...,...,...,...
2674,Vinci SA ADR (VCISY) [ST],S,04/21/2025,04/28/2025,"$1,001 - $15,000",S,ptr_pdfs\20030332.pdf,1001.0,15000.0
2675,Visa Inc. (V) [ST],S,04/21/2025,04/28/2025,"$1,001 - $15,000",S,ptr_pdfs\20030332.pdf,1001.0,15000.0
2676,Walmart Inc. Common Stock (WMT),S,04/21/2025,04/28/2025,"$1,001 - $15,000",S,ptr_pdfs\20030332.pdf,1001.0,15000.0
2677,Filing ID #20030336,None,None,None,None,NaN,ptr_pdfs\20030336.pdf,NaN,NaN


# **Loading into PostreSql**

In [20]:
df_cleaned.columns = [col.lower().replace(" ", "_") for col in df_cleaned.columns]

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

# Database connection parameters
user = "###############"
password = "############"      
host = "############"              
port = "5432"                   
database = "#################"

# Create SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")


In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql+psycopg2://postgres:################@#########:5432/###########"
)


In [ ]:
df_cleaned.to_sql(
    "ptr_transactions",
    engine,
    if_exists="replace",  
    index=False
)


679

In [24]:
engine.dispose()